## Predicting Flood Potential Based on Rain Fall for the San Lorenzo River Basin, California

ETL Step 1:  
- Extracting data from Stream Gage USGS 11160500 San Lorenzo at Big Trees for the time period 09/01/2014 to 09/01/2024.

  Stream Gage data was obtained through download via the [USGS for station 11160500]. River height in feet, and stream flow in cubic feet per second, measured every 15 minutes.

- Extracting precipitation data relative to the San Lorenzo River Watershed for the same time period, 09/01/2014 to 09/01/2024.
  Data was obtained from the [California Data Exchange Center, Department of Water Sources site]
  
  4 stations were identified that have hourly data readings in inches of rainfall:

| Location            | Code | Elevation | Latitude  | Longitude    | County     | Agency                              |
|---------------------|------|-----------|-----------|--------------|------------|-------------------------------------|
| BEN LOMOND (CDF)     | BLO  | 2630      | 37.132000 | -122.169998  | SANTA CRUZ | CA Dept of Forestry and Fire Protection |
| SCHULTIES RD         | SCH  | 1400      | 37.132999 | -121.969002  | SANTA CRUZ | Santa Cruz County                   |
| BOULDER CREEK        | BDC  | 800       | 37.141998 | -122.163002  | SANTA CRUZ | Santa Cruz County                   |
| BEN LOMOND           | BLN  | 365       | 37.092999 | -122.074997  | SANTA CRUZ | Santa Cruz County                   |

BLO is the only station that has continuous measurements, the other three stations have event based measurements that are recorded based on a tipping point.






[USGS for station 11160500]: https://waterdata.usgs.gov/monitoring-location/11160500/#parameterCode=00065&period=P7D&showMedian=false
[California Data Exchange Center, Department of Water Sources site]: https://cdec.water.ca.gov/dynamicapp/wsSensorData

In [21]:
# Import dependencies
import pandas as pd 

Extracting the Stream Gage data to a Pandas data frame

In [22]:
# Define the file path 
file_path = 'Resources/BigTrees11160500_9_2014_9_2024.txt'

# Skip the header rows and load the data into a DataFrame
stream = pd.read_csv(file_path, sep='\t', comment='#', skiprows=28, header=0)

# Rename the columns
stream.columns = ['agency', 'site_no', 'datetime', 'time_zone', 'gage_height', 'approval_code']

# Convert the 'datetime' column to datetime type for easier manipulation
stream['datetime'] = pd.to_datetime(stream['datetime'])

# Display the DataFrame
print(stream.head())

  agency   site_no            datetime time_zone  gage_height approval_code
0   USGS  11160500 2014-09-01 00:00:00       PDT         2.69             A
1   USGS  11160500 2014-09-01 00:15:00       PDT         2.69             A
2   USGS  11160500 2014-09-01 00:30:00       PDT         2.69             A
3   USGS  11160500 2014-09-01 00:45:00       PDT         2.69             A
4   USGS  11160500 2014-09-01 01:00:00       PDT         2.69             A


In [23]:
# Define the path for the new CSV output
output_csv = 'Resources/transformed_stream_gage_data.csv'

# Save the DataFrame to a new CSV file
stream.to_csv(output_csv, index=False)

print(f"Data has been saved to {output_csv}")

Data has been saved to Resources/transformed_stream_gage_data.csv


Transform date to have only one measurement per hour to match the rain data

In [24]:
# Convert the 'datetime' column to datetime format if it exists
if 'datetime' in stream.columns:
    stream['datetime'] = pd.to_datetime(stream['datetime'], errors='coerce')

# Extract the date and hour from the 'datetime' column
stream['date_hour'] = stream['datetime'].dt.floor('h')

# Group by the date and hour and get the max gage height for each hour
max_height_per_hour = stream.groupby('date_hour').agg({'gage_height': 'max'}).reset_index()

# Display the new DataFrame with the date, hour, and max height
print(max_height_per_hour.head())

# Define the path for the new CSV output
output_csv = 'Resources/transformed_stream_gage_data_hourly.csv'

# Save the DataFrame to a new CSV file
max_height_per_hour.to_csv(output_csv, index=False)

print(f"Data has been saved to {output_csv}")


            date_hour  gage_height
0 2014-09-01 00:00:00         2.69
1 2014-09-01 01:00:00         2.69
2 2014-09-01 02:00:00         2.69
3 2014-09-01 03:00:00         2.68
4 2014-09-01 04:00:00         2.68
Data has been saved to Resources/transformed_stream_gage_data_hourly.csv


Repeating the same process for the river flow data

In [25]:
# Define the file path 
file_path = 'Resources/BigTrees11160500_9_2014_9_2024_flow.txt'

# Skip the header rows and load the data into a DataFrame
# Skip rows with the description of column widths like '5s', '15s', etc.
stream_flow = pd.read_csv(file_path, sep='\t', comment='#', skiprows=[28, 29], header=0, dtype={'site_no': str, '14696_00060': float})

# Rename the columns for easier reference
stream_flow.columns = ['agency', 'site_no', 'datetime', 'time_zone', 'discharge', 'approval_code']

# Convert the 'datetime' column to datetime type for easier manipulation, ignoring errors
stream_flow['datetime'] = pd.to_datetime(stream['datetime'], errors='coerce')

# Display the DataFrame (first few rows)
print(stream_flow.head())

# Define the path for the new CSV output
output_csv = 'Resources/transformed_stream_discharge_data.csv'

# Save the DataFrame to a new CSV file
stream_flow.to_csv(output_csv, index=False)

print(f"Data has been saved to {output_csv}")


  agency   site_no            datetime time_zone  discharge approval_code
0   USGS  11160500 2014-09-01 00:00:00       PDT       7.09             A
1   USGS  11160500 2014-09-01 00:15:00       PDT       7.09             A
2   USGS  11160500 2014-09-01 00:30:00       PDT       7.09             A
3   USGS  11160500 2014-09-01 00:45:00       PDT       7.09             A
4   USGS  11160500 2014-09-01 01:00:00       PDT       7.09             A
Data has been saved to Resources/transformed_stream_discharge_data.csv


Get the Max flow value per hour

In [26]:
# Convert the 'datetime' column to datetime format if it exists
if 'datetime' in stream_flow.columns:
    stream_flow['datetime'] = pd.to_datetime(stream_flow['datetime'], errors='coerce')

# Extract the date and hour from the 'datetime' column
stream_flow['date_hour'] = stream_flow['datetime'].dt.floor('h')

# Group by the date and hour and get the max discharge for each hour
max_flow_per_hour = stream_flow.groupby('date_hour').agg({'discharge': 'max'}).reset_index()

# Display the new DataFrame with the date, hour, and max discharge
print(max_flow_per_hour.head())

# Define the path for the new CSV output
output_csv = 'Resources/transformed_max_flow_data_hourly.csv'

# Save the DataFrame to a new CSV file
max_flow_per_hour.to_csv(output_csv, index=False)

print(f"Data has been saved to {output_csv}")

            date_hour  discharge
0 2014-09-01 00:00:00       7.09
1 2014-09-01 01:00:00       7.09
2 2014-09-01 02:00:00       7.09
3 2014-09-01 03:00:00       6.86
4 2014-09-01 04:00:00       6.86
Data has been saved to Resources/transformed_max_flow_data_hourly.csv


Extracting rain data to Pandas data frame

In [66]:
# Load the Excel file
file_path = 'Resources/BLO_2.xlsx'

# Reading the Excel file to inspect sheet names and general structure
xls = pd.ExcelFile(file_path)

# Display the sheet names to understand how the data is organized
xls.sheet_names

/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['Sheet1']

In [68]:
# Since the file contains a single sheet 'Sheet1', let's load it and inspect the first few rows to understand the data structure.
rain_BLO = pd.read_excel(file_path, sheet_name='Sheet1')

# Drop the 'datetime' column
rain_BLO = rain_BLO.drop(columns=['DATE TIME'])

# Convert the 'obs_date' column to datetime
rain_BLO['OBS DATE'] = pd.to_datetime(rain['OBS DATE'], errors='coerce')

# Display the first few rows of the DataFrame to understand the structure
rain_BLO.head()

/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,OBS DATE,VALUE,DATA_FLAG,UNITS
0,BLO,H,2,RAIN,2014-09-01 00:00:00,0.04,,INCHES
1,BLO,H,2,RAIN,2014-09-01 01:00:00,0.04,,INCHES
2,BLO,H,2,RAIN,2014-09-01 02:00:00,0.04,,INCHES
3,BLO,H,2,RAIN,2014-09-01 03:00:00,0.04,,INCHES
4,BLO,H,2,RAIN,2014-09-01 04:00:00,0.04,,INCHES


In [69]:
# Define the path for the new CSV output
output_csv = 'Resources/transformed_BLO_rain_data_hourly.csv'

# Save the DataFrame to a new CSV file
rain.to_csv(output_csv, index=False)

print(f"Data has been saved to {output_csv}")

Data has been saved to Resources/transformed_BLO_rain_data_hourly.csv


In [70]:
print(rain_BLO.dtypes)

STATION_ID               object
DURATION                 object
SENSOR_NUMBER             int64
SENS_TYPE                object
OBS DATE         datetime64[ns]
VALUE                   float64
DATA_FLAG                object
UNITS                    object
dtype: object


Load and clean Rain data for stations BDC, BLN, and SCH that only record with tipping point and compare with Daily measured rainfall to check for errors  

A sensor type where the field measuring device uses a calibrated bucket that tips when full of precipitation. The amount of precipitation at which the device tips is usually 0.04 inches. There are some gages that tip at 0.01 inches. The value usually accumulates or gets larger until it is reset. A reset may occur if a technician visits the site or it is near the beginning of the season. The dates that designate a season varies according to different agencies (ie. July-June, October-September). Generally, this sensor type is used for real-time collection duration of hourly or event data.

BDC Station

In [56]:
import pandas as pd

# Load the two Excel files
file_16 = pd.read_excel('Resources/BDC_16.xlsx')
file_2 = pd.read_excel('Resources/BDC_2.xlsx')

# Convert 'OBS DATE' in file_16 to datetime and round to nearest hour
file_16['OBS DATE'] = pd.to_datetime(file_16['OBS DATE'])
file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')

# Calculate rainfall increments for file 16
file_16['Rainfall'] = file_16['VALUE'].diff().fillna(0)  # Calculate rainfall increments

# Calculate daily cumulative rainfall for File 16
file_16_daily = file_16.groupby(file_16['Rounded Date'].dt.date)['Rainfall'].sum().reset_index()
file_16_daily.columns = ['Date', 'Calculated Cumulative Rainfall (inches)']

# Convert 'OBS DATE' in file_2 to datetime
file_2['OBS DATE'] = pd.to_datetime(file_2['OBS DATE'])

# Extract relevant daily cumulative data from file 2
file_2_daily = file_2[['OBS DATE', 'VALUE']].dropna()
file_2_daily.columns = ['Date', 'Actual Cumulative Rainfall (inches)']

# Calculate daily rainfall for File 2
file_2_daily['Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)'].diff().fillna(0)

# Handle reset condition: if current day value is less than the previous day's value, it's a reset
file_2_daily.loc[file_2_daily['Actual Cumulative Rainfall (inches)'] < file_2_daily['Actual Cumulative Rainfall (inches)'].shift(1), 
                 'Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)']

# Ensure both 'Date' columns are of the same type
file_2_daily['Date'] = file_2_daily['Date'].dt.date

# Merge the two datasets on 'Date'
merged_data = pd.merge(file_16_daily, file_2_daily[['Date', 'Daily Rainfall (inches)']], on='Date', how='inner')

# Step 4: Identify discrepancies
merged_data['Discrepancy'] = merged_data['Calculated Cumulative Rainfall (inches)'] != merged_data['Daily Rainfall (inches)']

# Display the merged data with calculated discrepancies
print(merged_data)

merged_data.to_csv('Resources/BDC_Daily_Rainfall_compare.csv', index=False)


/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


            Date  Calculated Cumulative Rainfall (inches)  \
0     2015-10-23                                      0.0   
1     2015-10-24                                      0.0   
2     2015-10-25                                      0.0   
3     2015-10-26                                      0.0   
4     2015-10-27                                      0.0   
...          ...                                      ...   
2629  2024-08-16                                      0.0   
2630  2024-08-19                                      0.0   
2631  2024-08-23                                      0.0   
2632  2024-08-26                                      0.0   
2633  2024-08-31                                      0.0   

      Daily Rainfall (inches)  Discrepancy  
0                         0.0        False  
1                         0.0        False  
2                         0.0        False  
3                         0.0        False  
4                         0.0        False

/var/folders/_d/q6dl733j0hdfhxbg883rg3xc0000gn/T/ipykernel_2662/1403092099.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')


Remove data from 16 file if discrepancy is greater than .25 inches

In [61]:
# Calculate the absolute difference between calculated and actual rainfall
merged_data['Difference'] = abs(merged_data['Calculated Cumulative Rainfall (inches)'] - merged_data['Daily Rainfall (inches)'])

# Filter out days where the difference is greater than 0.25 inches
filtered_data = merged_data[merged_data['Difference'] <= 0.25]

# Get the dates to keep from the filtered dataset
dates_to_keep = filtered_data['Date'].unique()

# Convert 'OBS DATE' in file_16 to datetime and ensure it's compatible for filtering
file_16['Date'] = file_16['OBS DATE'].dt.date

# Filter the original 16 file to keep only rows that match the dates in 'dates_to_keep'
filtered_file_16 = file_16[file_16['Date'].isin(dates_to_keep)].copy()  # .copy() to avoid SettingWithCopyWarning

# Now, calculate cumulative hourly rainfall for each day in the filtered 16 file
filtered_file_16['Cumulative Daily Rainfall'] = filtered_file_16.groupby('Date')['Rainfall'].cumsum()

# Merge the daily rainfall back into the filtered file
daily_totals = filtered_file_16.groupby('Date')['Rainfall'].sum().reset_index()
daily_totals.columns = ['Date', 'Total Daily Rainfall']

# Merge the daily totals into the filtered 16 file
filtered_file_16 = pd.merge(filtered_file_16, daily_totals, on='Date')

# Drop the additional 'Date' column after filtering if needed
filtered_file_16 = filtered_file_16.drop(columns=['Date'])

# Set 'Rainfall' and 'Cumulative Daily Rainfall' to 0 if 'Total Daily Rainfall' is 0
filtered_file_16.loc[filtered_file_16['Total Daily Rainfall'] == 0, ['Rainfall', 'Cumulative Daily Rainfall']] = 0

# Save the result to a CSV file
filtered_file_16.to_csv('Resources/BDC_cleaned_hourly_Rainfall.csv', index=False)


# Display the final dataset with cumulative hourly rainfall and total daily rainfall
print(filtered_file_16)



      STATION_ID DURATION  SENSOR_NUMBER SENS_TYPE         DATE TIME  \
0            BDC        E             16   RAINTIP  2015-10-23 00:52   
1            BDC        E             16   RAINTIP  2015-10-23 12:52   
2            BDC        E             16   RAINTIP  2015-10-24 00:52   
3            BDC        E             16   RAINTIP  2015-10-25 00:52   
4            BDC        E             16   RAINTIP  2015-10-25 12:52   
...          ...      ...            ...       ...               ...   
11429        BDC        E             16   RAINTIP  2024-08-19 01:49   
11430        BDC        E             16   RAINTIP  2024-08-19 13:49   
11431        BDC        E             16   RAINTIP  2024-08-23 13:49   
11432        BDC        E             16   RAINTIP  2024-08-26 13:49   
11433        BDC        E             16   RAINTIP  2024-08-31 13:49   

                 OBS DATE  VALUE DATA_FLAG   UNITS        Rounded Date  \
0     2015-10-23 00:52:00   0.31            INCHES 2015-10-23

BLN Station

In [62]:
import pandas as pd

# Load the two Excel files
file_16 = pd.read_excel('Resources/BLN_16.xlsx')
file_2 = pd.read_excel('Resources/BLN_2.xlsx')

# Convert 'OBS DATE' in file_16 to datetime and round to nearest hour
file_16['OBS DATE'] = pd.to_datetime(file_16['OBS DATE'])
file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')

# Calculate rainfall increments for file 16
file_16['Rainfall'] = file_16['VALUE'].diff().fillna(0)  # Calculate rainfall increments

# Calculate daily cumulative rainfall for File 16
file_16_daily = file_16.groupby(file_16['Rounded Date'].dt.date)['Rainfall'].sum().reset_index()
file_16_daily.columns = ['Date', 'Calculated Cumulative Rainfall (inches)']

# Convert 'OBS DATE' in file_2 to datetime
file_2['OBS DATE'] = pd.to_datetime(file_2['OBS DATE'])

# Extract relevant daily cumulative data from file 2
file_2_daily = file_2[['OBS DATE', 'VALUE']].dropna()
file_2_daily.columns = ['Date', 'Actual Cumulative Rainfall (inches)']

# Calculate daily rainfall for File 2
file_2_daily['Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)'].diff().fillna(0)

# Handle reset condition: if current day value is less than the previous day's value, it's a reset
file_2_daily.loc[file_2_daily['Actual Cumulative Rainfall (inches)'] < file_2_daily['Actual Cumulative Rainfall (inches)'].shift(1), 
                 'Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)']

# Ensure both 'Date' columns are of the same type
file_2_daily['Date'] = file_2_daily['Date'].dt.date

# Merge the two datasets on 'Date'
merged_data = pd.merge(file_16_daily, file_2_daily[['Date', 'Daily Rainfall (inches)']], on='Date', how='inner')

# Step 4: Identify discrepancies
merged_data['Discrepancy'] = merged_data['Calculated Cumulative Rainfall (inches)'] != merged_data['Daily Rainfall (inches)']

# Display the merged data with calculated discrepancies
print(merged_data)

merged_data.to_csv('Resources/BLN_Daily_Rainfall_compare.csv', index=False)


/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


            Date  Calculated Cumulative Rainfall (inches)  \
0     2015-10-16                                      0.0   
1     2015-10-17                                      0.0   
2     2015-10-18                                      0.0   
3     2015-10-19                                      0.0   
4     2015-10-20                                      0.0   
...          ...                                      ...   
2296  2024-08-25                                      0.0   
2297  2024-08-26                                      0.0   
2298  2024-08-27                                      0.0   
2299  2024-08-29                                      0.0   
2300  2024-08-30                                      0.0   

      Daily Rainfall (inches)  Discrepancy  
0                         0.0        False  
1                         0.0        False  
2                         0.0        False  
3                         0.0        False  
4                         0.0        False

/var/folders/_d/q6dl733j0hdfhxbg883rg3xc0000gn/T/ipykernel_2662/2137034109.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')


Remove data from 16 file if discrepancy is greater than .25 inches

In [63]:
# Calculate the absolute difference between calculated and actual rainfall
merged_data['Difference'] = abs(merged_data['Calculated Cumulative Rainfall (inches)'] - merged_data['Daily Rainfall (inches)'])

# Filter out days where the difference is greater than 0.25 inches
filtered_data = merged_data[merged_data['Difference'] <= 0.25]

# Get the dates to keep from the filtered dataset
dates_to_keep = filtered_data['Date'].unique()

# Convert 'OBS DATE' in file_16 to datetime and ensure it's compatible for filtering
file_16['Date'] = file_16['OBS DATE'].dt.date

# Filter the original 16 file to keep only rows that match the dates in 'dates_to_keep'
filtered_file_16 = file_16[file_16['Date'].isin(dates_to_keep)].copy()  # .copy() to avoid SettingWithCopyWarning

# Now, calculate cumulative hourly rainfall for each day in the filtered 16 file
filtered_file_16['Cumulative Daily Rainfall'] = filtered_file_16.groupby('Date')['Rainfall'].cumsum()

# Merge the daily rainfall back into the filtered file
daily_totals = filtered_file_16.groupby('Date')['Rainfall'].sum().reset_index()
daily_totals.columns = ['Date', 'Total Daily Rainfall']

# Merge the daily totals into the filtered 16 file
filtered_file_16 = pd.merge(filtered_file_16, daily_totals, on='Date')

# Drop the additional 'Date' column after filtering if needed
filtered_file_16 = filtered_file_16.drop(columns=['Date'])

# Set 'Rainfall' and 'Cumulative Daily Rainfall' to 0 if 'Total Daily Rainfall' is 0
filtered_file_16.loc[filtered_file_16['Total Daily Rainfall'] == 0, ['Rainfall', 'Cumulative Daily Rainfall']] = 0

# Save the result to a CSV file
filtered_file_16.to_csv('Resources/BLN_cleaned_hourly_Rainfall.csv', index=False)


# Display the final dataset with cumulative hourly rainfall and total daily rainfall
print(filtered_file_16)



      STATION_ID DURATION  SENSOR_NUMBER SENS_TYPE         DATE TIME  \
0            BLN        E             16   RAINTIP  2015-10-16 11:08   
1            BLN        E             16   RAINTIP  2015-10-16 23:08   
2            BLN        E             16   RAINTIP  2015-10-17 11:08   
3            BLN        E             16   RAINTIP  2015-10-17 23:08   
4            BLN        E             16   RAINTIP  2015-10-18 11:08   
...          ...      ...            ...       ...               ...   
10600        BLN        E             16   RAINTIP  2024-08-25 12:03   
10601        BLN        E             16   RAINTIP  2024-08-26 12:03   
10602        BLN        E             16   RAINTIP  2024-08-27 12:03   
10603        BLN        E             16   RAINTIP  2024-08-29 00:03   
10604        BLN        E             16   RAINTIP  2024-08-30 12:03   

                 OBS DATE  VALUE DATA_FLAG   UNITS        Rounded Date  \
0     2015-10-16 11:08:00   0.00            INCHES 2015-10-16

SCH Station

In [64]:
import pandas as pd

# Load the two Excel files
file_16 = pd.read_excel('Resources/SCH_16.xlsx')
file_2 = pd.read_excel('Resources/SCH_2.xlsx')

# Convert 'OBS DATE' in file_16 to datetime and round to nearest hour
file_16['OBS DATE'] = pd.to_datetime(file_16['OBS DATE'])
file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')

# Calculate rainfall increments for file 16
file_16['Rainfall'] = file_16['VALUE'].diff().fillna(0)  # Calculate rainfall increments

# Calculate daily cumulative rainfall for File 16
file_16_daily = file_16.groupby(file_16['Rounded Date'].dt.date)['Rainfall'].sum().reset_index()
file_16_daily.columns = ['Date', 'Calculated Cumulative Rainfall (inches)']

# Convert 'OBS DATE' in file_2 to datetime
file_2['OBS DATE'] = pd.to_datetime(file_2['OBS DATE'])

# Extract relevant daily cumulative data from file 2
file_2_daily = file_2[['OBS DATE', 'VALUE']].dropna()
file_2_daily.columns = ['Date', 'Actual Cumulative Rainfall (inches)']

# Calculate daily rainfall for File 2
file_2_daily['Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)'].diff().fillna(0)

# Handle reset condition: if current day value is less than the previous day's value, it's a reset
file_2_daily.loc[file_2_daily['Actual Cumulative Rainfall (inches)'] < file_2_daily['Actual Cumulative Rainfall (inches)'].shift(1), 
                 'Daily Rainfall (inches)'] = file_2_daily['Actual Cumulative Rainfall (inches)']

# Ensure both 'Date' columns are of the same type
file_2_daily['Date'] = file_2_daily['Date'].dt.date

# Merge the two datasets on 'Date'
merged_data = pd.merge(file_16_daily, file_2_daily[['Date', 'Daily Rainfall (inches)']], on='Date', how='inner')

# Step 4: Identify discrepancies
merged_data['Discrepancy'] = merged_data['Calculated Cumulative Rainfall (inches)'] != merged_data['Daily Rainfall (inches)']

# Display the merged data with calculated discrepancies
print(merged_data)

merged_data.to_csv('Resources/SCH_Daily_Rainfall_compare.csv', index=False)


/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/Caskroom/miniforge/base/envs/flooding_model/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


            Date  Calculated Cumulative Rainfall (inches)  \
0     2016-10-01                                      0.0   
1     2016-10-02                                      0.0   
2     2016-10-03                                      0.0   
3     2016-10-04                                      0.0   
4     2016-10-05                                      0.0   
...          ...                                      ...   
2713  2024-08-22                                      0.0   
2714  2024-08-23                                      0.0   
2715  2024-08-25                                      0.0   
2716  2024-08-30                                      0.0   
2717  2024-08-31                                      0.0   

      Daily Rainfall (inches)  Discrepancy  
0                         0.0        False  
1                         0.0        False  
2                         0.0        False  
3                         0.0        False  
4                         0.0        False

/var/folders/_d/q6dl733j0hdfhxbg883rg3xc0000gn/T/ipykernel_2662/458426343.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  file_16['Rounded Date'] = file_16['OBS DATE'].dt.round('H')


Remove data from 16 file if discrepancy is greater than .25 inches

In [65]:
# Calculate the absolute difference between calculated and actual rainfall
merged_data['Difference'] = abs(merged_data['Calculated Cumulative Rainfall (inches)'] - merged_data['Daily Rainfall (inches)'])

# Filter out days where the difference is greater than 0.25 inches
filtered_data = merged_data[merged_data['Difference'] <= 0.25]

# Get the dates to keep from the filtered dataset
dates_to_keep = filtered_data['Date'].unique()

# Convert 'OBS DATE' in file_16 to datetime and ensure it's compatible for filtering
file_16['Date'] = file_16['OBS DATE'].dt.date

# Filter the original 16 file to keep only rows that match the dates in 'dates_to_keep'
filtered_file_16 = file_16[file_16['Date'].isin(dates_to_keep)].copy()  # .copy() to avoid SettingWithCopyWarning

# Now, calculate cumulative hourly rainfall for each day in the filtered 16 file
filtered_file_16['Cumulative Daily Rainfall'] = filtered_file_16.groupby('Date')['Rainfall'].cumsum()

# Merge the daily rainfall back into the filtered file
daily_totals = filtered_file_16.groupby('Date')['Rainfall'].sum().reset_index()
daily_totals.columns = ['Date', 'Total Daily Rainfall']

# Merge the daily totals into the filtered 16 file
filtered_file_16 = pd.merge(filtered_file_16, daily_totals, on='Date')

# Drop the additional 'Date' column after filtering if needed
filtered_file_16 = filtered_file_16.drop(columns=['Date'])

# Set 'Rainfall' and 'Cumulative Daily Rainfall' to 0 if 'Total Daily Rainfall' is 0
filtered_file_16.loc[filtered_file_16['Total Daily Rainfall'] == 0, ['Rainfall', 'Cumulative Daily Rainfall']] = 0

# Save the result to a CSV file
filtered_file_16.to_csv('Resources/SCH_cleaned_hourly_Rainfall.csv', index=False)


# Display the final dataset with cumulative hourly rainfall and total daily rainfall
print(filtered_file_16)



      STATION_ID DURATION  SENSOR_NUMBER SENS_TYPE         DATE TIME  \
0            SCH        E             16   RAINTIP  2016-10-01 01:47   
1            SCH        E             16   RAINTIP  2016-10-01 13:47   
2            SCH        E             16   RAINTIP  2016-10-02 01:47   
3            SCH        E             16   RAINTIP  2016-10-02 13:47   
4            SCH        E             16   RAINTIP  2016-10-03 01:47   
...          ...      ...            ...       ...               ...   
13719        SCH        E             16   RAINTIP  2024-08-22 12:23   
13720        SCH        E             16   RAINTIP  2024-08-23 12:23   
13721        SCH        E             16   RAINTIP  2024-08-25 12:23   
13722        SCH        E             16   RAINTIP  2024-08-30 12:25   
13723        SCH        E             16   RAINTIP  2024-08-31 12:25   

                 OBS DATE  VALUE DATA_FLAG   UNITS        Rounded Date  \
0     2016-10-01 01:47:00  43.17            INCHES 2016-10-01

Clean Data, check for missing values

In [31]:
# Load the files
file1_path = 'Resources/transformed_max_flow_data_hourly.csv'
file2_path = 'Resources/transformed_BLO_rain_data_hourly.csv'
file3_path = 'Resources/transformed_stream_gage_data_hourly.csv'

# Read the CSV files
max_flow_data = pd.read_csv(file1_path)
rain_data_BLO = pd.read_csv(file2_path)
stream_gage_data = pd.read_csv(file3_path)

# Check for missing date times in each file
missing_max_flow = max_flow_data[max_flow_data.isnull().any(axis=1)]
missing_rain_data_BLO = rain_data_BLO[rain_data_BLO.isnull().any(axis=1)]
missing_stream_gage = stream_gage_data[stream_gage_data.isnull().any(axis=1)]

# Display any rows with missing date times
(missing_max_flow, missing_rain_data_BLO, missing_stream_gage)


(Empty DataFrame
 Columns: [date_hour, discharge]
 Index: [],
       STATION_ID DURATION  SENSOR_NUMBER SENS_TYPE             OBS DATE  \
 1076         BLO        H              2      RAIN  2014-10-15 20:00:00   
 1077         BLO        H              2      RAIN  2014-10-15 21:00:00   
 1078         BLO        H              2      RAIN  2014-10-15 22:00:00   
 1079         BLO        H              2      RAIN  2014-10-15 23:00:00   
 1080         BLO        H              2      RAIN  2014-10-16 00:00:00   
 ...          ...      ...            ...       ...                  ...   
 82615        BLO        H              2      RAIN  2024-02-03 16:00:00   
 82788        BLO        H              2      RAIN  2024-02-10 21:00:00   
 82801        BLO        H              2      RAIN  2024-02-11 10:00:00   
 83465        BLO        H              2      RAIN  2024-03-10 03:00:00   
 84071        BLO        H              2      RAIN  2024-04-04 09:00:00   
 
        VALUE DATA_FLAG 

This tells me only the rain dataset is missing data

In [32]:
# Count the number of missing data rows in the rain dataset
missing_rain_data_count_BLO = rain_data_BLO['VALUE'].isnull().sum()

missing_rain_data_count_BLO


230

Analyze rain data around missing values. Am I missing crucial data?  
I know from the USGS site all of the historic gage heights that have been recorded. The greatest heights recorded during the project time period have been saved into Resources/Max_crest_data.csv

In [34]:
# Filter for all the days that have any missing rain values
missing_days_rain_BLO = rain_data_BLO[rain_data_BLO['VALUE'].isnull()]['OBS DATE'].unique()

# Create a DataFrame with the missing days
missing_days_BLO_df = pd.DataFrame(missing_days_rain_BLO, columns=['OBS DATE'])

# Save the missing days to a CSV file
missing_days_output_path = 'Resources/missing_rain_days_BLO.csv'
missing_days_BLO_df.to_csv(missing_days_output_path, index=False)

missing_days_output_path


'Resources/missing_rain_days.csv'

Compare missing data days with Max Crest Days

In [36]:
# Load the two files
max_crest_data = pd.read_csv('Resources/Max_crest_data.csv')
missing_rain_days_BLO_data = pd.read_csv('Resources/missing_rain_days_BLO.csv')

# Convert both date columns to datetime
max_crest_data['Date'] = pd.to_datetime(max_crest_data['Date'], errors='coerce')
missing_rain_days_BLO_data['OBS DATE'] = pd.to_datetime(missing_rain_days_BLO_data['OBS DATE'], errors='coerce')

# Compare and find matching dates between the two datasets
matching_dates = max_crest_data[max_crest_data['Date'].isin(missing_rain_days_BLO_data['OBS DATE'])]

matching_dates


,Date,Height (ft)


Check if any missing dates fall within 3 days (before or after) of the max crest dates

In [38]:
# Use timedelta to create a 3-day window for comparison
three_days_window = pd.Timedelta(days=3)

# Compare the missing rain days and max crest days to see if they fall within 3 days of each other
matching_dates_within_3_days = []

for missing_date in missing_rain_days_BLO_data['OBS DATE']:
    for crest_date in max_crest_data['Date']:
        if abs(missing_date - crest_date) <= three_days_window:
            matching_dates_within_3_days.append({'Missing Date': missing_date, 'Crest Date': crest_date})

# Convert the results to a DataFrame
matching_within_3_days_df = pd.DataFrame(matching_dates_within_3_days)

matching_within_3_days_df

,Missing Date,Crest Date
0,2017-01-03 13:00:00,2017-01-04
1,2017-02-09 10:00:00,2017-02-07
2,2023-03-12 03:00:00,2023-03-10
3,2023-03-12 03:00:00,2023-03-14


In [45]:
# First, let's count the number of missing values per day in the missing rain data
missing_counts_per_day = missing_rain_days_BLO_data['OBS DATE'].dt.date.value_counts()

# Identify days where all 24 hours are missing and days where more than 3 values are missing
completely_missing_days = missing_counts_per_day[missing_counts_per_day == 24]
more_than_3_missing_days = missing_counts_per_day[missing_counts_per_day > 3]

# Create DataFrames for both completely missing days and more than 3 missing days
completely_missing_days_df = pd.DataFrame(completely_missing_days).reset_index().rename(columns={'index': 'OBS Date', 'OBS Date': 'Missing Count'})
more_than_3_missing_days_df = pd.DataFrame(more_than_3_missing_days).reset_index().rename(columns={'index': 'OBS DATE', 'OBS Date': 'Missing Count'})

more_than_3_missing_days_df

,OBS DATE,count
0,2021-07-22,24
1,2014-10-16,24
2,2015-03-19,12
3,2023-09-18,12
4,2023-12-07,11
5,2024-01-30,10
6,2023-12-09,10
7,2014-10-17,9
8,2021-04-18,8
9,2021-07-23,8


## EDA

AttributeError: Can only use .dt accessor with datetimelike values